## Generación base de datos el Parque Nacional Terepaima

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

In [1]:
import rioxarray 
import xarray

import geopandas
from pyproj.crs import CRS

import pandas as pd

import os

Cambiando directorio de trabajo

In [2]:
print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/gee-metview/terepaima/code
> Directorio actual:  /media/javier/Compartida/doctorado/gee-metview/terepaima


### Proyección

In [3]:
precipitacion_crs = CRS.from_wkt('GEOGCS["Coordinate System imported from GRIB file",DATUM["unnamed",SPHEROID["Sphere",6367470,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]')

### Polígonos

In [4]:
park_poligono = geopandas.read_file('./polygons/park/terepaima.shp')
parque_rectangulo = geopandas.read_file('./polygons/rectangle/rectangulo_terepaima.shp')

### Paths

In [5]:
path_precipitacion = './cdsapi/precipitacion_regrilla.nc'
path_elevacion = './SRTMGL3/elevacion_regrilla.nc'
path_ndvi = './ndvi/ndvi_regrilla.nc'

### Precipitación

In [6]:
precipitacion_rds = rioxarray.open_rasterio(path_precipitacion, masked=True)
precipitacion_rds

<xarray.DataArray 'precipitacion_mm' (time: 629, y: 3, x: 3)>
array([[[1.698088, 1.375303, 1.234049],
        [2.647525, 2.170181, 1.748822],
        [3.840237, 2.96101 , 2.36513 ]],

       [[0.460838, 0.417095, 0.406519],
        [0.946488, 0.863066, 0.733055],
        [1.542819, 1.352538, 1.17916 ]],

       ...,

       [[4.349987, 3.714037, 3.251037],
        [5.167873, 4.310113, 3.775549],
        [6.709513, 5.151767, 4.38548 ]],

       [[1.41659 , 1.525311, 1.747265],
        [2.436019, 2.318654, 2.456253],
        [4.195963, 3.350959, 3.250625]]])
Coordinates:
  * time         (time) int64 719163 719194 719222 ... 738215 738246 738276
  * x            (x) float64 -69.38 -69.28 -69.18
  * y            (y) float64 9.96 9.86 9.76
    spatial_ref  int64 0
Attributes:
    add_offset:             0.0
    GRIB_COMMENT:           Total precipitation [m]
    GRIB_ELEMENT:           TP
    GRIB_FORECAST_SECONDS:  86400
    GRIB_REF_TIME:          0
    GRIB_SHORT_NAME:        0-SFC
    GRIB_UNIT:              [m]
    GRIB_VALID_TIME:        86400
    long_name:              0[-] SFC (Ground or water surface)
    scale_factor:           1.0

In [7]:
def id_point_format(x,y):
  """
  Funcion para identificar el id del centroide
  """

  #--
  if x == -69.38 and y==9.96:
    id_point = 1
  elif x == -69.28 and y==9.96:
    id_point = 2
  elif x == -69.18 and y==9.96:
    id_point = 3
  #--
  elif x == -69.38 and y==9.86:
    id_point = 4
  elif x == -69.28 and y==9.86:
    id_point = 5
  elif x == -69.18 and y==9.86:
    id_point = 6
  #--
  elif x == -69.38 and y==9.76:
    id_point = 7
  elif x == -69.28 and y==9.76:
    id_point = 8
  elif x == -69.18 and y==9.76:
    id_point = 9
  else:
    id_point = None

  return id_point

In [8]:
columns_precipitacion = ['time',	'x',	'y', 'precipitacion_mm']
pd_precipitacion = precipitacion_rds.to_dataframe()\
                                    .reset_index()[columns_precipitacion]

pd_precipitacion['time'] = pd_precipitacion['time'].astype(int)
pd_precipitacion['x'] = pd_precipitacion['x'].astype(float).round(6)
pd_precipitacion['y'] = pd_precipitacion['y'].astype(float).round(6)
pd_precipitacion['id_point'] = pd_precipitacion[['x','y']].round(2).apply(lambda x: id_point_format(x=x.x,y=x.y), 1 ).astype(int)
pd_precipitacion['precipitacion_mm'] = pd_precipitacion['precipitacion_mm'].astype(float)

pd_precipitacion = pd_precipitacion.dropna()

pd_precipitacion.head(10)

,time,x,y,precipitacion_mm,id_point
0,719163,-69.38,9.96,1.698088,1
1,719163,-69.28,9.96,1.375303,2
2,719163,-69.18,9.96,1.234049,3
3,719163,-69.38,9.86,2.647525,4
4,719163,-69.28,9.86,2.170181,5
5,719163,-69.18,9.86,1.748822,6
6,719163,-69.38,9.76,3.840237,7
7,719163,-69.28,9.76,2.961010,8
8,719163,-69.18,9.76,2.365130,9
9,719194,-69.38,9.96,0.460838,1


In [9]:
from datetime import datetime

print(datetime.fromordinal(pd_precipitacion.time.min()))
print(datetime.fromordinal(pd_precipitacion.time.max()))

1970-01-01 00:00:00
2022-05-01 00:00:00


### Elevación

In [10]:
elevacion_rds = rioxarray.open_rasterio(path_elevacion, masked=True)
elevacion_rds

<xarray.Dataset>
Dimensions:            (y: 3, x: 3, band: 1)
Coordinates:
  * y                  (y) float64 9.96 9.86 9.76
  * x                  (x) float64 -69.38 -69.28 -69.18
  * band               (band) int64 1
    spatial_ref        int64 0
Data variables:
    elevacion_media    (band, y, x) float32 ...
    elevacion_mediana  (band, y, x) float32 ...
    elevacion_maxima   (band, y, x) float32 ...
Attributes:
    Conventions:  CF-1.7
    history:      2022-08-20T13:00 GRIB to CDM+CF via cfgrib-0.9.10.1/ecCodes...

In [11]:
columns_elevacion = ['x',	'y', 'elevacion_media','elevacion_mediana','elevacion_maxima']
pd_elevacion = elevacion_rds.to_dataframe()\
                            .reset_index()[columns_elevacion]

pd_elevacion['x'] = pd_elevacion['x'].astype(float).round(6)
pd_elevacion['y'] = pd_elevacion['y'].astype(float).round(6)
pd_elevacion['elevacion_media'] = pd_elevacion['elevacion_media'].astype(float)
pd_elevacion['elevacion_mediana'] = pd_elevacion['elevacion_mediana'].astype(float)
pd_elevacion['elevacion_maxima'] = pd_elevacion['elevacion_maxima'].astype(float)

pd_elevacion.head(10)

,x,y,elevacion_media,elevacion_mediana,elevacion_maxima
0,-69.38,9.96,1001.651794,986.0,1632.0
1,-69.28,9.96,896.212402,875.0,1556.0
2,-69.18,9.96,470.032227,462.0,755.0
3,-69.38,9.86,1273.258179,1264.0,1922.0
4,-69.28,9.86,1118.260376,1172.0,1622.0
5,-69.18,9.86,415.874298,385.0,826.0
6,-69.38,9.76,946.540039,904.0,1811.0
7,-69.28,9.76,779.210327,780.0,1376.0
8,-69.18,9.76,319.055603,280.0,863.0


### NDVI

In [12]:
ndvi_rds = rioxarray.open_rasterio(path_ndvi, masked=True)
ndvi_rds

<xarray.Dataset>
Dimensions:       (y: 3, x: 3, time: 112)
Coordinates:
  * y             (y) float64 9.96 9.86 9.76
  * x             (x) float64 -69.38 -69.28 -69.18
  * time          (time) int64 734503 734534 734563 ... 738246 738276 738307
    spatial_ref   int64 0
Data variables:
    ndvi_media    (time, y, x) float32 ...
    ndvi_mediana  (time, y, x) float32 ...
    ndvi_maxima   (time, y, x) float32 ...
Attributes:
    Conventions:  CF-1.7
    history:      2022-08-20T13:00 GRIB to CDM+CF via cfgrib-0.9.10.1/ecCodes...

In [13]:
columns_ndvi = ['time', 'x',	'y', 'ndvi_media','ndvi_mediana','ndvi_maxima']
pd_ndvi = ndvi_rds.to_dataframe()\
                  .reset_index()[columns_ndvi]

pd_ndvi['time'] = pd_ndvi['time'].astype(int)
pd_ndvi['x'] = pd_ndvi['x'].astype(float).round(6)
pd_ndvi['y'] = pd_ndvi['y'].astype(float).round(6)
pd_ndvi['ndvi_media'] = pd_ndvi['ndvi_media'].astype(float)
pd_ndvi['ndvi_mediana'] = pd_ndvi['ndvi_mediana'].astype(float)
pd_ndvi['ndvi_maxima'] = pd_ndvi['ndvi_maxima'].astype(float)

pd_ndvi.head(10)

,time,x,y,ndvi_media,ndvi_mediana,ndvi_maxima
0,734503,-69.38,9.96,NaN,NaN,NaN
1,734534,-69.38,9.96,NaN,NaN,NaN
2,734563,-69.38,9.96,NaN,NaN,NaN
3,734594,-69.38,9.96,NaN,NaN,NaN
4,734624,-69.38,9.96,NaN,NaN,NaN
5,734655,-69.38,9.96,NaN,NaN,NaN
6,734685,-69.38,9.96,NaN,NaN,NaN
7,734716,-69.38,9.96,NaN,NaN,NaN
8,734747,-69.38,9.96,NaN,NaN,NaN
9,734777,-69.38,9.96,NaN,NaN,NaN


#### Integrando Bases

In [14]:
from datetime import datetime

pd_integracion = pd.merge(pd_precipitacion, pd_ndvi, on = ['time','x','y'], how='left')
pd_integracion = pd.concat([ pd_integracion, pd_ndvi[pd_ndvi.time > pd_precipitacion.time.max()] ])\
                   .merge(pd_elevacion, on = ['x','y'], how='left')\
                   .rename(columns={"x": "longitud", "y": "latitud"})

pd_integracion = pd_integracion[pd_integracion.id_point.notna()]
pd_integracion['id_point'] = pd_integracion['id_point'].astype(int)
pd_integracion['time_actualizacion'] = int(datetime.today().toordinal())
pd_integracion['park'] = 'terepaima'

pd_integracion.head(10)

,time,longitud,latitud,precipitacion_mm,id_point,ndvi_media,ndvi_mediana,ndvi_maxima,elevacion_media,elevacion_mediana,elevacion_maxima,time_actualizacion,park
0,719163,-69.38,9.96,1.698088,1,NaN,NaN,NaN,1001.651794,986.0,1632.0,738387,terepaima
1,719163,-69.28,9.96,1.375303,2,NaN,NaN,NaN,896.212402,875.0,1556.0,738387,terepaima
2,719163,-69.18,9.96,1.234049,3,NaN,NaN,NaN,470.032227,462.0,755.0,738387,terepaima
3,719163,-69.38,9.86,2.647525,4,NaN,NaN,NaN,1273.258179,1264.0,1922.0,738387,terepaima
4,719163,-69.28,9.86,2.170181,5,NaN,NaN,NaN,1118.260376,1172.0,1622.0,738387,terepaima
5,719163,-69.18,9.86,1.748822,6,NaN,NaN,NaN,415.874298,385.0,826.0,738387,terepaima
6,719163,-69.38,9.76,3.840237,7,NaN,NaN,NaN,946.540039,904.0,1811.0,738387,terepaima
7,719163,-69.28,9.76,2.961010,8,NaN,NaN,NaN,779.210327,780.0,1376.0,738387,terepaima
8,719163,-69.18,9.76,2.365130,9,NaN,NaN,NaN,319.055603,280.0,863.0,738387,terepaima
9,719194,-69.38,9.96,0.460838,1,NaN,NaN,NaN,1001.651794,986.0,1632.0,738387,terepaima


In [15]:
pd_integracion.groupby(['longitud','latitud']).count()

time  precipitacion_mm  id_point  ndvi_media  ndvi_mediana  \
longitud latitud                                                               
-69.38   9.76      629               629       629           0             0   
         9.86      629               629       629         111           111   
         9.96      629               629       629           0             0   
-69.28   9.76      629               629       629         111           111   
         9.86      629               629       629         111           111   
         9.96      629               629       629         111           111   
-69.18   9.76      629               629       629         111           111   
         9.86      629               629       629         111           111   
         9.96      629               629       629           0             0   

                  ndvi_maxima  elevacion_media  elevacion_mediana  \
longitud latitud                                                    
-69.38   9.76               0              629                629   
         9.86             111              629                629   
         9.96               0              629                629   
-69.28   9.76             111              629                629   
         9.86             111              629                629   
         9.96             111              629                629   
-69.18   9.76             111              629                629   
         9.86             111              629                629   
         9.96               0              629                629   

                  elevacion_maxima  time_actualizacion  park  
longitud latitud                                              
-69.38   9.76                  629                 629   629  
         9.86                  629                 629   629  
         9.96                  629                 629   629  
-69.28   9.76                  629                 629   629  
         9.86                  629                 629   629  
         9.96                  629                 629   629  
-69.18   9.76                  629                 629   629  
         9.86                  629                 629   629  
         9.96                  629                 629   629

In [16]:
pd_integracion.groupby(['time']).count()

,longitud,latitud,precipitacion_mm,id_point,ndvi_media,ndvi_mediana,ndvi_maxima,elevacion_media,elevacion_mediana,elevacion_maxima,time_actualizacion,park
time,,,,,,,,,,,,
719163,9,9,9,9,0,0,0,9,9,9,9,9
719194,9,9,9,9,0,0,0,9,9,9,9,9
719222,9,9,9,9,0,0,0,9,9,9,9,9
719253,9,9,9,9,0,0,0,9,9,9,9,9
719283,9,9,9,9,0,0,0,9,9,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...
738156,9,9,9,9,6,6,6,9,9,9,9,9
738187,9,9,9,9,6,6,6,9,9,9,9,9
738215,9,9,9,9,6,6,6,9,9,9,9,9


In [17]:
print(datetime.fromordinal(pd_integracion.time.min()))
print(datetime.fromordinal(pd_integracion.time.max()))

1970-01-01 00:00:00
2022-05-01 00:00:00


In [18]:
from datetime import datetime

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month + 1

diff_month( datetime.fromordinal(pd_integracion.time.max()),
                             datetime.fromordinal(pd_integracion.time.min()))

629

In [19]:
import json

json_data = '{ "data":' + pd_integracion.to_json(orient="records") +\
                ',"park" : "terepaima"' + "}"

with open('./data/json_data.json', 'w') as outfile:
    outfile.write(json_data)

In [20]:
# Mongo coleccion
documentos = json.loads( pd_integracion.to_json(orient="records") )

documentos[-1]

{'time': 738276,
 'longitud': -69.18,
 'latitud': 9.76,
 'precipitacion_mm': 3.2506245188,
 'id_point': 9,
 'ndvi_media': 0.7451670766,
 'ndvi_mediana': 0.7790499926,
 'ndvi_maxima': 0.8840999603,
 'elevacion_media': 319.0556030273,
 'elevacion_mediana': 280.0,
 'elevacion_maxima': 863.0,
 'time_actualizacion': 738387,
 'park': 'terepaima'}

### Conexión MONGODB

Cambiando directorio

In [21]:
print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/gee-metview/terepaima
> Directorio actual:  /media/javier/Compartida/doctorado/gee-metview


In [22]:
# Configuracion
import yaml

# Definiendo variables
with open('./config.yml') as stream:
    config = yaml.safe_load(stream)

In [23]:
import pymongo

username = config['MONGO_USER']
password = config['MONGO_PASSWORD']
cluster = config['MONGO_CLUSTER']

conn_str = f"mongodb+srv://{username}:{password}@{cluster}.wsg1gnp.mongodb.net/?retryWrites=true&w=majority"
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)

In [24]:
# Creando base de datos
db = client['SSEV']
db.name

'SSEV'

In [25]:
# insertando coleccion
coleccion = db['meteorological']

In [26]:
# coleccion.create_index([("time", pymongo.DESCENDING), 
#                         ("park", pymongo.DESCENDING)],
#                         background=True)

In [27]:
# Insertando documentos
for doc in documentos:
  coleccion.update_one({"time":doc.get('time'),"park":"terepaima"}, {"$set":doc}, upsert=True)